# Coursework 3: Bilinear Inverse Problems and Low-Rank Matrix Recovery

[] By tick the checkbox, we hereby declare that this coursework report is our own and autonomous work. We have acknowledged all material and sources used in its preparation, including books, articles, reports, lecture notes, internet software packages, and any other kind of document, electronic or personal communication. This work has not been submitted for any other assessment.

## 3.1 Test Data Generation (10%)

We consider the low-rank matrix completion problem given by 
$$
    \bm{y} = \mathcal{P}_{\Omega}(\bm{X}) 
$$
where $\bm{X} \in \mathbb{R}^{m \times n}$ is a low rank matrix of rank $r$. 

Data generation: Write $\bm{X} = \bm{U} \bm{G} \bm{V}^{\mathsf{T}}$, where $\bm{U} \in \mathbb{R}^{m \times r}$, $\bm{G} \in \mathbb{R}^{r \times r}$, and $\bm{V} \in \mathbb{R}^{n \times r}$ are matrices with i.i.d. $\mathcal{N}(0,1)$ Gaussian entries. (Note that by $\bm{X} = \bm{U} \bm{G} \bm{V}^{\mathsf{T}}$ we are not talking about SVD.)


Design and implement a function `LRMC_data_gen` to generate test data. Provide necessary documentation.

In [1]:
using LinearAlgebra
using Random
using Distributions 
using Plots
using StatsBase

function GaussianGen(m::Int,n::Int)

    A = randn(m,n)
    for i = 1:n
        v = A[:,i]
        A[:,i] = v/norm(v,2)
    end
    return A
end


function Observation_samples(X::AbstractArray, m::Int64, n::Int64, samples::Int64)

    random_indices = sample(randperm(m*n), samples, replace=false)

    Ω = zeros(Bool, m, n)
    Ω[random_indices] .= true

    Y = zeros(m, n)

    Y[Ω] = X[Ω]

    return Y, Ω
end

function LRMC_data_gen(m::Int64, n::Int64, r::Int64)
    U = GaussianGen(m, r)
    G = GaussianGen(r, r)
    V = GaussianGen(n, r)

    X = U * G * V'

    return X
end

LRMC_data_gen (generic function with 1 method)

## 3.2 Matrix Completion Techniques

In the following, the suggested simulation setup is that $m = 32$, $n=48$, $r$ varies in $2:2:8$, and $|\Omega|/mn$ varies in $\{1/8,~ 1/6,~ 1/4,~ 1/2\}$. 

### 3.2.1 Alternating Minimization (20%)

Design, implement, and run tests for the alternating minimization method for low-rank matrix completion. Use the function name `LRMCRec_AM`. Provide necessary documentation.

### 3.2.2 Iterative Hard Thresholding (IHT) (20%)

Design, implement, and run simple tests for the IHT algorithm for low-rank matrix completion. Use the function name `LRMCRec_IHT`. Provide necessary documentation. 

In [73]:

function r_rank_approx(M,r::Int)
    # Gives a r rank approximation to the input matrix M 
    F = svd(M)
    U = F.U ; V = F.V ; Sigma = F.S
    r_rank_M = Sigma[1]*U[:,1]*transpose(V[:,1])
    for i = 2:r
        u = U[:,i]; v = V[:,i]; s = Sigma[i]
        r_rank_M += s * u * transpose(v)
    end
    return r_rank_M
end 

function LRMCRec_IHT(Y, Ω, r::Int, τ::Float64)
    num_iters = 0  ; error = 0 ;
    m = size(Y,1)  ; n = size(Y,2)
    X = zeros(m,n) ; temp = zeros(m,n);   
    error_list = []; 
    
    while num_iters < 1000 
        # Iteration algorithm to reduce objective error 
        temp[Ω] = Y[Ω] - X[Ω]
        X  = r_rank_approx(X + τ*(temp),r)
        

        #Calculate objective error
        error = norm(Y[Ω] - X[Ω],2)^2
        append!(error_list,error)
        
        #Increase the number of iterations
        num_iters = num_iters + 1

    end 
    return X ,1:num_iters, error_list
end

LRMCRec_IHT (generic function with 1 method)

In [75]:
m = 48; n = 32
Ω_sizes = [192,256,384,768]
τ = 1.0
plot_arr = Plots.Plot{Plots.GRBackend}[]

for r= 2:2:8 
    p_r = plot()
    X  = LRMC_data_gen(m, n, 8)
    for Ω_size in Ω_sizes 
        Y, Ω = Observation_samples(X, m, n, Ω_size)
        X_final, num_iters,error_list = LRMCRec_IHT(Y,Ω,r,τ)
        plot!(p_r, num_iters, error_list, label="|Ω|=$(Ω_size)", title="IHT for r=$(r)",fmt = :png,dpi=2080)
        @show rank(X_final)
    end
    push!(plot_arr,p_r)
end

plot(plot_arr[1],plot_arr[2],plot_arr[3],plot_arr[4])


rank(X_final) = 2
rank(X_final) = 2
rank(X_final) = 2
rank(X_final) = 2
rank(X_final) = 4
rank(X_final) = 4
rank(X_final) = 4
rank(X_final) = 4
rank(X_final) = 6
rank(X_final) = 6
rank(X_final) = 6
rank(X_final) = 6
rank(X_final) = 8
rank(X_final) = 8
rank(X_final) = 8
rank(X_final) = 8


### 3.2.3 Iterative Shrinkage-Thresholding Algorithm (ISTA) (25%)

Design, implement, and run simple tests for ISTA (to solve the Lasso formulation) for low-rank matrix completion. Use the function name `LRMCRec_ISTA`. Provide necessary documentation. Use simulations to discuss the choice of parameters.

### 3.2.4 Lasso-ADMM (25%)

Design, implement, and run simple tests for an ADMM algorithm (to solve the Lasso formulation) for low-rank matrix completion. Use the function name `LRMCRec_ADMM`. Provide necessary documentation. Compare ADMM and ISTA in terms of convergence.

## Highlight

Please list a couple of highlights of your coursework that may impress your markers.